In [4]:
import glob
import os
import operator
import tqdm
import pandas as pd
from ete3 import Tree as eTree
import numpy as np
import dendropy as dpy

# from tree_distance import PhyloTree
from Bio import SeqIO

import dendropy
from dendropy import Tree
from dendropy.calculate import treecompare

from scipy.spatial import distance

OUTPUT_FOLDER_REF = "/Volumes/RECHERCHE/FAC/FBM/DBC/cdessim2/read2tree/D2c/wally/r2t/r2t_aling_sp_removal/ref"
OUTPUT_FOLDER = "/Volumes/RECHERCHE/FAC/FBM/DBC/cdessim2/read2tree/D2c/r2t/mouse/r2t/rm_species/"


# arath
#t_neighbour = ['ARATH','ARALY','ARAAL','BRAOL','LOTJA','GOSHI','VITVI','SOLTU']

# yeast
#t_neighbour = ['YEAST','CANGA','DEBHA','DEKBR','TUBMM','SCHPO','SPIPN']

# mouse
t_neighbour = ['MOUSE','RATNO','CHICK','XENTR','PETMA','BRAFL','STRPU']

tns = dendropy.TaxonNamespace()
out_dict = {}
translate_names = {'pac20X':20, 'pac10X':10, 'pac02X':0.2, 'pac1X':1, 'pac5X':5, 'pac05X':0.5,'ill20X':20, 'ill10X':10, 'ill02X':0.2, 'ill1X':1, 'ill2X':2, 'ill5X':5, 'ill05X':0.5, 'nan20X':20, 'nan10X':10, 'nan02X':0.2, 'nan1X':1, 'nan5X':5, 'nan05X':0.5, 'nan50X':50, 'nan100X':100}

In [5]:
df_trees = pd.read_csv('/Users/daviddylus/Downloads/benchmark_reads - like_shen-2.csv')
df_trees

,species,coverage,dist,assembly_time,technology,ref trees,r2t tree shen,r2t tree,Unnamed: 8
0,arath,0.2,0,NaN,Illumina,NaN,"(CHLVA:0.3296090866,((SELML:0.1971417168,(((((...","(BRARP:0.00539097,((((((((((((((TRIUA:0.019465...","(VITVI:0.09177319336826902,(((((((ORYLO:0.0175..."
1,arath,0.2,1,NaN,Illumina,NaN,"(SELML:0.2002209391,((CHLVA:0.3438467287,KLEFL...","(ARATH:0.0345819,((BRARP:0.00537901,(BRANA:0.0...","(VITVI:0.09177761921655042,(((((((ORYLO:0.0175..."
2,arath,0.2,2,NaN,Illumina,NaN,"(SORBI:0.0139614912,(((((((((((BRARP:0.0041331...","(TRIUA:0.0194671,(((((((((((((BRARP:0.00533048...","(VITVI:0.09179352450169354,(((((((ORYLO:0.0175..."
3,arath,0.2,3,NaN,Illumina,NaN,"(3_ill:0.0704052773,((((CHLVA:0.3267546679,KLE...","(ARATH:0.100639,((((((((((((TRIUA:0.0194609,AE...","(VITVI:0.09141184952100921,(((((((ORYLO:0.0175..."
4,arath,0.2,4,NaN,Illumina,NaN,"(4_ill:0.0654262303,((((CHLVA:0.3459067888,KLE...","(ARATH:0.0983816,(((((((((((TRIUA:0.019467,AEG...","(VITVI:0.08986284816204641,(((((((ORYLO:0.0175..."
...,...,...,...,...,...,...,...,...,...
373,yeast,20.0,2,20774.0,PacBIO,"(SALR5:0.2724975652,(((((((BOTFB:0.1370638124,...","(SPIPN:0.2299730648,(((SALR5:0.3076006667,MONB...","(YEAST:0.151916,(((((SALR5:0.333326,MONBE:0.31...","(SALR5:0.2511666572314431,((((((((TALSN:0.1166..."
374,yeast,20.0,3,20774.0,PacBIO,"(BOTFB:0.1438173545,(((COCLU:0.1828847018,AURP...","(TALSN:0.1347920486,(((((((CAPO3:0.3709647034,...","(MONBE:0.316357,((((YEAST:0.214991,((((TALSN:0...","(SALR5:0.2589124419037486,(((((((TALSN:0.12013..."
375,yeast,20.0,4,20774.0,PacBIO,"(YEAST:0.4007114048,(((MONBE:0.2850049111,SALR...","(4_pac:0.1387700228,((((MONBE:0.2974460978,SAL...","(MONBE:0.314583,((((YEAST:0.162328,SCHPO:0.309...","(SALR5:0.29433779029474216,(((YEAST:0.48484088..."
376,yeast,20.0,5,20774.0,PacBIO,"(SALR5:0.3037354340,(((SPIPN:0.2072047071,BATD...","(MONBE:0.2945619465,(((5_pac:0.1670709379,(BAT...","(CREFR:0.464086,(SALR5:0.333488,MONBE:0.314554...","(SALR5:0.299908542030726,(((YEAST:0.6104814961..."


In [6]:
def get_p_dist_vec(tree, species):
    pd_vec = {}
    pd_vec[species] = 0.0
    ref_sp = tree.taxon_namespace.get_taxon(species)
    pdm = tree.phylogenetic_distance_matrix()
    dist_to_ref = []
    for leaf in tree.leaf_nodes():
        if leaf.taxon is not ref_sp:
#             pd_vec.append(pdm.patristic_distance(ref_sp, leaf.taxon))
            pd_vec[leaf.taxon.label] = pdm.patristic_distance(ref_sp, leaf.taxon)
    #out = [x[1][0] for x in sorted(pd_vec.items())]
    return pd_vec

def get_ref_tree_dic(folder,suffix):
    out_dict = {}
    for file in glob.glob(os.path.join(folder,suffix+'*.contree')):
        basename = int(os.path.basename(file).split("_")[2].rsplit('.phy.contree')[0])
        t_tmp = eTree(file)
        tree = Tree()
        tree1 = tree.get_from_string(t_tmp.write(),"newick",taxon_namespace=tns)
        out_dict[str(basename)] = tree1
    return out_dict

def get_dist_closest_neighbour(tree, species):
    my_dict = get_p_dist_vec(tree, species)
    sorted_x = sorted(my_dict.items(), key=operator.itemgetter(1))
    species = [x[0] for x in sorted_x]
    pd_dist = [x[1] for x in sorted_x]
    out_dict = {'species':species, 'pd_dist':pd_dist}
    df = pd.DataFrame(data=out_dict)
    return df

In [7]:
from collections import OrderedDict
tree_dic = get_ref_tree_dic(OUTPUT_FOLDER_REF,"mouse")
print(tree_dic['6'])
tree_dic

(MEGSC:0.185739,((((((SCHMA:0.464225,((CAEEL:0.240267,PRIPA:0.228538)100:0.211522,TRISP:0.394223)100:0.10184)100:0.0780451,((((AMPQE:0.385165,TRIAD:0.338095)100:0.0811786,STRPU:0.271884)100:0.03624,((CIOIN:0.077624,CIOSA:0.0900199)100:0.280118,MOUSE:0.229932)100:0.0587112)100:0.0441898,(CAPI1:0.231871,(CRAGI:0.1886,LOTGI:0.189335)100:0.0626604)100:0.059973)100:0.0575288)100:0.111879,ACYPI:0.305016)100:0.044689,ATTCE:0.216739)100:0.0371392,TRICA:0.199755)100:0.104024,AEDAE:0.160884)100:0.0614335,(DROME:0.0642664,DROMO:0.0628691)100:0.11456)


{'3': <Tree object at 0x15bcf2bf0>,
 '6': <Tree object at 0x1097dfbe0>,
 '5': <Tree object at 0x15bd34910>,
 '2': <Tree object at 0x15bd361a0>,
 '4': <Tree object at 0x15bd37c70>,
 '1': <Tree object at 0x15bd557e0>,
 '0': <Tree object at 0x15bd57b80>}

In [8]:
my_dict_pd = OrderedDict(sorted(get_p_dist_vec(tree_dic['0'], 'MOUSE').items(), key=lambda t: t[1]))
my_dict_pd

OrderedDict([('MOUSE', 0.0),
             ('RATNO', 0.0228721),
             ('HUMAN', 0.06235472),
             ('PONAB', 0.06664676),
             ('NOMLE', 0.07054645),
             ('CAVPO', 0.07272700000000001),
             ('OTOGA', 0.07635395),
             ('CHICK', 0.16579410000000003),
             ('ANOCA', 0.1825749),
             ('XENTR', 0.2279361),
             ('PETMA', 0.38514879999999996),
             ('BRAFL', 0.5555659),
             ('CIOIN', 0.6646889),
             ('CIOSA', 0.6770838),
             ('STRPU', 0.6835551),
             ('CAPI1', 0.7076815999999999),
             ('CRAGI', 0.7265119),
             ('LOTGI', 0.7278909),
             ('TRIAD', 0.8379704000000001),
             ('ATTCE', 0.8542981000000001),
             ('TRICA', 0.8737256),
             ('AMPQE', 0.8860754000000002),
             ('ACYPI', 0.8991569000000001),
             ('AEDAE', 0.9387066000000002),
             ('DROMO', 1.0170405000000002),
             ('DROME', 1.018298000

## Tree distances vs reference Tree

### Change the species name to something that is possible to compare with reference

e.g. nan or ill containig leaf name to CANVA or ARATH

In [9]:
trees = []
same_neighbour = []
dist_to_neighbour = []
ref_trees = []
name = []
mean = []
stdev = []
gcov = []
num_ogs = []
species_freq = []
species = []
method = []
euc = []
normalized_euc = []
normalized_euc1 = []
rf = []
nrf = []
wrf = []
gcov = []
names = []
pd_dif = []
branch_diff = []
pd_cor = []
pd_euc = []
internode_dist = []
closest_neighbour_dist = []

In [11]:
for idist, tmp_tree in tqdm.tqdm(tree_dic.items()):
    ref_tree = tmp_tree
    ref_dict_pd = OrderedDict(sorted(get_p_dist_vec(ref_tree, 'MOUSE').items(), key=lambda t: t[1]))
#     print(ref_dict_pd)
    ref_neighbour = list(ref_dict_pd.items())[1]
#     print(tmp_tree)
    pd_ref_dist = [x[1] for x in sorted(get_p_dist_vec(ref_tree, 'MOUSE').items(), key=operator.itemgetter(1))]
    pd_vec_ref = np.array(pd_ref_dist)
    df_trees_sub = df_trees[(df_trees['dist']==int(idist)) & (df_trees['species'] == 'MOUSE'.lower())]
    for i, r in df_trees_sub.iterrows():
        t_tmp = eTree(r['r2t tree'])
        tree = Tree()
        print(t_tmp)
        do_it = False
        for leaf in t_tmp.traverse():
            if 'ONT' in leaf.name:
                leaf.name = 'MOUSE'
                meth = 'ONT'
                do_it = True
            elif 'Illumina' in leaf.name:
                leaf.name = 'MOUSE'
                meth = 'Illumina'
                do_it = True
            elif 'PacBIO' in leaf.name:
                leaf.name = 'MOUSE'
                meth = 'PacBIO'
                do_it = True
        if do_it:
            tree1 = tree.get_from_string(t_tmp.write(),"newick",taxon_namespace=tns)
            test_dict_pd = OrderedDict(sorted(get_p_dist_vec(tree1, 'MOUSE').items(), key=lambda t: t[1]))
#             print(test_dict_pd)
            test_neighbour = list(test_dict_pd.items())[1]
            if ref_neighbour[0] in test_neighbour[0]:
                same_neighbour.append('same')
            else:
                same_neighbour.append('different')
            dist_to_neighbour.append(ref_neighbour[1]-test_dict_pd[ref_neighbour[0]])
            trees.append(str(tree1)+";")
            ref_trees.append(str(ref_tree)+";")
            gcov.append(r['coverage']) # gcov = given coverage
            species.append('MOUSE')
            method.append(meth)
            internode_dist.append(int(idist))
            closest_neighbour_dist.append(my_dict_pd[t_neighbour[int(idist)]])
            tree1.encode_bipartitions()
            euc.append(treecompare.euclidean_distance(tree1, ref_tree))
            normalized_euc1.append(treecompare.euclidean_distance(tree1, ref_tree)/(tree1.length()+ref_tree.length()))
            rf.append(treecompare.symmetric_difference(tree1, ref_tree))
            wrf.append(treecompare.weighted_robinson_foulds_distance(tree1, ref_tree))
            pd_dist = [x[1] for x in sorted(get_p_dist_vec(tree1, 'MOUSE').items(), key=operator.itemgetter(1))]
            pd_sp = np.array(pd_dist)
#                     pd_cor.append(np.corrcoef(pd_vec_ref,pd_sp)[0][1])
            pd_cor.append(0)
#                     pd_euc.append(distance.euclidean(pd_vec_ref,pd_sp))
            pd_euc.append(0)
            et = eTree(str(ref_tree)+";")
            et2 = eTree(str(tree1)+";")
            res = et.compare(et2,unrooted=True)
            nrf.append(res["norm_rf"])

            # normalize tree edge lengths such that the maximum distance between root and leaf is 1
            max_dist_to_root = np.array(tree1.calc_node_root_distances()).max()
            for edge in tree1.postorder_edge_iter():
                if edge.length is None:
                    edge.length = 0
                else:
                    edge.length = float(edge.length)/max_dist_to_root

            max_dist_to_root = np.array(ref_tree.calc_node_root_distances()).max()
            for edge in ref_tree.postorder_edge_iter():
                if edge.length is None:
                    edge.length = 0
                else:
                    edge.length = float(edge.length)/max_dist_to_root

            normalized_euc.append(treecompare.euclidean_distance(tree1, ref_tree))


# out_dict = {'species':species, 'closest_neighbour_dist':closest_neighbour_dist, 'internode_dist':internode_dist, 'method':method, 'gcov':gcov, 'trees':trees,'rf':rf, 'nrf':nrf, 'wrf':wrf, 'euc':euc, 'neuc1':normalized_euc1, 'neuc':normalized_euc, 'pd_cor':pd_cor, 'pd_euc':pd_euc}
out_dict = {'same_neighbour':same_neighbour, 'dist_to_neighbour':dist_to_neighbour, 'species':species, 'closest_neighbour_dist':closest_neighbour_dist, 'internode_dist':internode_dist, 'method':method, 'gcov':gcov, 'trees':trees, 'ref_trees':ref_trees, 'rf':rf, 'nrf':nrf, 'wrf':wrf, 'euc':euc, 'neuc1':normalized_euc1, 'neuc':normalized_euc}

df = pd.DataFrame(data=out_dict)
df
# df.sort_values(by=['gcov'])

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:00<00:00, 33.62it/s]


   /-MOUSE
  |
  |--XENTR
  |
  |                                 /-MEGSC
  |                              /-|
  |                             |  |   /-DROME
  |                           /-|   \-|
  |                          |  |      \-DROMO
  |                        /-|  |
  |                       |  |   \-AEDAE
  |                     /-|  |
  |                    |  |   \-TRICA
  |                  /-|  |
  |                 |  |   \-ATTCE
  |                 |  |
  |                 |   \-ACYPI
  |               /-|
  |              |  |         /-PRIPA
  |              |  |      /-|
--|              |  |   /-|   \-CAEEL
  |              |  |  |  |
  |            /-|   \-|   \-TRISP
  |           |  |     |
  |           |  |      \-SCHMA
  |           |  |
  |           |  |   /-CAPI1
  |           |   \-|
  |         /-|     |   /-CRAGI
  |        |  |      \-|
  |        |  |         \-LOTGI
  |        |  |
  |        |  |      /-AMPQE
  |      /-|  |   /-|
  |     |  |   

,same_neighbour,dist_to_neighbour,species,closest_neighbour_dist,internode_dist,method,gcov,trees,ref_trees,rf,nrf,wrf,euc,neuc1,neuc


In [27]:
df.to_csv("/Users/daviddylus/Desktop/r2t_align_mouse_sp_removal-like_shen.csv", index=False)

In [ ]:
t_yeast_neighbour

In [ ]:
for key, val in out_dict.items():
    print(key)
    print(len(val))

In [28]:
df


,closest_neighbour_dist,dist_to_neighbour,euc,gcov,internode_dist,method,neuc,neuc1,nrf,ref_trees,rf,same_neighbour,species,trees,wrf
0,0.000000,-0.008479,0.122963,10,0,PacBIO,0.144248,0.009283,0.000000,"(NOMLE:0.0128472,((((((((((((((((MEGSC:0.18573...",0,same,MOUSE,"(XENTR:0.0939019,((((((TRIAD:0.310929,AMPQE:0....",0.727701
1,0.000000,-0.006760,0.022482,20,0,PacBIO,0.029277,0.001792,0.000000,"(NOMLE:0.011563428721509457,((((((((((((((((ME...",0,same,MOUSE,"(MOUSE:0.0206187,(((((((((TRIAD:0.311596,AMPQE...",0.117172
2,0.000000,-0.010468,0.023248,5,0,PacBIO,0.024153,0.001853,0.000000,"(NOMLE:0.011563428721509457,((((((((((((((((ME...",0,same,MOUSE,"(DROMO:0.0516206,((((((((TRISP:0.352866,(CAEEL...",0.114348
3,0.000000,-0.009771,0.023373,10,0,ONT,0.030858,0.001861,0.000000,"(NOMLE:0.011563428721509457,((((((((((((((((ME...",0,same,MOUSE,"(MOUSE:0.0235231,(((((((((((((((DROMO:0.051745...",0.115606
4,0.000000,-0.009857,0.024081,20,0,ONT,0.031289,0.001917,0.000000,"(NOMLE:0.011563428721509457,((((((((((((((((ME...",0,same,MOUSE,"(MOUSE:0.0235245,((((PETMA:0.166156,(((((TRIAD...",0.117518
5,0.000000,-0.010584,0.024473,5,0,ONT,0.031978,0.001951,0.000000,"(NOMLE:0.011563428721509457,((((((((((((((((ME...",0,same,MOUSE,"(MOUSE:0.024009,(((((((((((((((DROMO:0.0516962...",0.120438
6,0.227936,0.064093,0.113376,10,3,PacBIO,0.238570,0.008942,0.000000,"(PETMA:0.185834,((((((((((MEGSC:0.185767,(DROM...",0,same,MOUSE,"(DROMO:0.0513506,((((((((((((MOUSE:0.0604565,X...",0.634596
7,0.227936,0.064036,0.224220,20,3,PacBIO,0.200884,0.016887,0.000000,"(PETMA:0.2028205894429042,((((((((((MEGSC:0.20...",0,same,MOUSE,"(TRISP:0.36059,((((((((DROMO:0.0511165,DROME:0...",1.252028
8,0.227936,0.068000,0.220488,5,3,PacBIO,0.087096,0.016575,0.000000,"(PETMA:0.20282058944290424,((((((((((MEGSC:0.2...",0,same,MOUSE,"(TRIAD:0.318071,(((((PETMA:0.162814,(MOUSE:0.0...",1.227479
9,0.227936,0.061187,0.221733,10,3,ONT,0.199034,0.016685,0.000000,"(PETMA:0.20282058944290424,((((((((((MEGSC:0.2...",0,same,MOUSE,"(TRISP:0.360816,((((((TRIAD:0.317523,AMPQE:0.3...",1.240650
